In [1]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
print(load_dotenv())
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)
logging.langsmith("yong_12")

True
[API KEY]
sk-None-FZcW7iODyxjiAh7qYSpIT3BlbkFJyqnfN***************
LangSmith 추적을 시작합니다.
[프로젝트명]
yong_12


In [2]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
prompt = hub.pull("lawwu/chain_of_density")

# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.1)
    | json_parser
)

In [4]:
chain.invoke(content)

[{'Missing_Entities': 'Optuna',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization library for machine learning. Optuna provides a simple and efficient way to search for the best hyperparameters for a given model. It uses a variety of search algorithms to explore the hyperparameter space and find the optimal values. With Optuna, you can easily tune the hyperparameters of your machine learning models and improve their performance.'},
 {'Missing_Entities': 'trial.suggest_categorical()',
  'Denser_Summary': 'Optuna provides a wide range of functions to suggest hyperparameters, including trial.suggest_categorical(). This function allows you to specify a categorical parameter and its possible choices. Optuna will then explore the different choices and find the best one for your model. By using trial.suggest_categorical(), you can easily optimize categorical hyperparameters and improve the performance of your machine learning models.'},
 {'Missing_Entities': 't

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json

loader = WebBaseLoader(
    "https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content

class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 데이터사이언스, 머신러닝, 인공지능에 대한 개념을 설명하고 있습니다. 박정현 서울대 EPM 연구원이 작성한 이 글은 이러한 기술들이 우리 생활에 어떻게 적용되고 있는지, 그리고 이 분야를 공부하고자 하는 사람들에게 어떤 조언을 할 수 있는지에 대해 다루고 있습니다. 또한, 데이터사이언스가 가장 넓은 범위를 가지고 있으며, 머신러닝과 인공지능이 어떻게 서로 관련되어 있는지에 대해서도 설명하고 있습니다. 이 글은 이 분야에 대한 기본적인 이해를 돕기 위해 여러 가지 예시와 함께 이론적, 실용적 측면에서 접근하고 있습니다."
    },
    {
        "Missing_Entities": "데이터사이언스 대학원; 인공지능 국가전략; 튜링테스트",
        "Denser_Summary": "박정현 서울대 EPM 연구원은 데이터사이언스, 머신러닝, 인공지능의 개념과 실생활 적용 예를 설명합니다. 데이터사이언스가 가장 넓은 범위를 가지며, 여러 대학에서 데이터사이언스 대학원을 설립하고, 인공지능 국가전략이 발표됨을 언급합니다. 머신러닝과 인공지능의 관계, 튜링테스트를 통한 인공지능의 이해 방법도 다룹니다. 이 글은 이론적, 실용적 측면에서 이 분야에 대한 기본적인 이해를 돕고자 합니다."
    },
    {
        "Missing_Entities": "지도학습; 비지도 학습; 클러스터링",
        "Denser_Summary": "박정현 서울대 EPM 연구원은 데이터사이언스, 머신러닝, 인공지능의 정의와 적용을 설명하며, 데이터사이언스 대학원 설립, 인공지능 국가전략 발표를 언급합니다. 머신러닝의 지도학습, 비지도 학습, 클러스터링 방법론과 튜링테스트를 통한 인공지능 이해를 다룹니다. 이 글은 이론적, 실용적 측면에서 이 분야의 기본적인 이해를 목표로 합니다."
    },
    {
      